In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import OrderedDict
import time
from sklearn.metrics import mean_squared_error,roc_auc_score,mean_absolute_error,log_loss
import sys
sys.path.append('../benchmark/')
from lvxnn_test import lvxnn
from xgb_test import xgb
from svd_test import svd
from deepfm_fm_test import deepfm_fm
import sys
sys.path.append('../../')
from lvxnn.LVXNN import LV_XNN
from lvxnn.DataReader import data_initialize

import tensorflow as tf
tf.compat.v1.enable_eager_execution()
random_state=0
train= pd.read_csv('../movielens/data/train.csv')
test = pd.read_csv('../movielens/data/test.csv')

task_type = "Regression"

list1 = train.columns
meta_info = OrderedDict()
for i in list1:
    meta_info[i]={'type': 'categorical','source':'item'} 
meta_info['Occupation']={"type":"categorical",'source':'user'}
meta_info['Gender']={"type":"categorical",'source':'user'}
meta_info['Age'] = {"type":"continues",'source':'user'}
meta_info['user_id']={"type":"id",'source':'user'}
meta_info['item_id']={"type":"id",'source':'item'}
meta_info['target']={"type":"target",'source':''}

lx_params = {
        "main_effect_epochs":50,
        "interaction_epochs" : 50 ,
        "tuning_epochs" : 50 , 
        "mf_training_iters": 200,
        "u_group_num":50,
        "i_group_num":50
    }

tr_x, tr_Xi, tr_y, tr_idx, te_x, te_Xi, te_y, val_x, val_Xi, val_y, val_idx, meta_info, model_info = data_initialize(train,test,meta_info,task_type ,'warm', random_state)

W0701 08:33:36.298127 22428 deprecation.py:323] From C:\Users\64161\Anaconda3\lib\site-packages\tensorflow_core\python\compat\v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


Memory usage of dataframe is 146.52 MB
Memory usage after optimization is: 19.84 MB
Decreased by 86.5%
Memory usage of dataframe is 36.63 MB
Memory usage after optimization is: 4.96 MB
Decreased by 86.5%
test cold start user: 0
test cold start item: 34
validation cold start user: 0
validation cold start item: 51


In [2]:
result_lvxnn = lvxnn('warm',tr_x, val_x, te_x, tr_y, val_y, te_y,tr_Xi, val_Xi, te_Xi, tr_idx, val_idx, meta_info, model_info, task_type , random_state=0, params=lx_params)
result_svd = svd('warm',tr_x, val_x, te_x, tr_y, val_y, te_y,tr_Xi, val_Xi, te_Xi, tr_idx, val_idx, meta_info, model_info, task_type , random_state=0)
result_deepfm, result_fm = deepfm_fm('warm',train,test,tr_x, val_x, te_x, tr_y, val_y, te_y,tr_Xi, val_Xi, te_Xi, tr_idx, val_idx, meta_info, model_info, task_type , random_state=0, epochs=50)
result_xgb = xgb('warm',tr_x, val_x, te_x, tr_y, val_y, te_y,tr_Xi, val_Xi, te_Xi, tr_idx, val_idx, meta_info, model_info, task_type , random_state=0)

result_ml = pd.concat([result_lvxnn,result_svd,result_xgb,result_deepfm,result_fm],0)

result_ml.to_csv('movielens_result.csv',index=None)

E0701 08:33:57.400531 22428 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



ListWrapper(['Gender', 'Age', 'Occupation', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
Traceback (most recent call last):
  File "C:\Users\64161\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-a8538efc6341>", line 1, in <module>
    result_lvxnn = lvxnn('warm',tr_x, val_x, te_x, tr_y, val_y, te_y,tr_Xi, val_Xi, te_Xi, tr_idx, val_idx, meta_info, model_info, task_type , random_state=0, params=lx_params)
  File "../benchmark\lvxnn_test.py", line 38, in lvxnn
    model.fit(tr_x, val_x, tr_y, val_y, tr_Xi, val_Xi, tr_idx, val_idx)
  File "../..\lvxnn\LVXNN.py", line 170, in fit
    verbose=self.verbose, val_ratio=self.val_ratio, early_stop_thres=self.early_stop_thres,random_state=self.random_state)
  File "../..\lvxnn\g

KeyboardInterrupt: 